In [1]:
import pandas as pd
import numpy as np

In [2]:
path="EquiposHUV-Zona4-copia.xlsx"
df=pd.read_excel(path)

In [3]:
df_salida=pd.DataFrame()
columnas_originales=df[["Nombre","Descripcion adicional","Marca","Modelo","Estado actual","Servicio","Clasificación biomedica","Clasificación de riesgo", "Cantidad de correctivos registrados"]]
df_salida=columnas_originales.copy()

In [4]:
df_salida["Estado ponderado"]=np.where(df_salida["Estado actual"]=="Activo",1,np.where(df_salida["Estado actual"]=="Fuera de servicio",1,0))


In [5]:
path2="datos/servicios.xlsx"
df_servicios=pd.read_excel(path2)
df_servicios.head()

,Servicio,Peso
0,UCI 2,3.0
1,MEDICA HOMBRES,2.0
2,MEDICA MUJERES,2.0
3,UCI,3.0
4,CIRUGIA PLASTICA,3.0


In [6]:
servicios_dic = df_servicios.set_index('Servicio').to_dict()["Peso"]
df_salida["Servicio ponderado"]=df_salida["Servicio"].apply(lambda x: servicios_dic[x])
df_salida.head()

,Nombre,Descripcion adicional,Marca,Modelo,Estado actual,Servicio,Clasificación biomedica,Clasificación de riesgo,Cantidad de correctivos registrados,Estado ponderado,Servicio ponderado
0,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,Fuera de servicio,QUIROFANO 11,DIAGNOSTICO,IIB,0,1,3.0
1,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIA,0,1,3.0
2,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIB,0,1,3.0
3,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Fuera de servicio,QUIROFANO 1- NEURO,DIAGNOSTICO,IIB,0,1,3.0
4,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,Fuera de servicio,QUIROFANO 2- NEURO,DIAGNOSTICO,IIB,1,1,3.0


In [23]:
clasificacion_dic={"TRATAMIENTO Y MANTENIMIENTO DE LA VIDA":3,
                    "RAHABILITACIÓN":2,"DIAGNOSTICO":2,
                    "APOYO":1,"N/R":1,"ANÁLISIS DE LABORATORIO":1,"REHABILITACIÓN":1,"NaN":1}
riesgo_dic={"I":1,"IIA":2,"IIB":3,"III":4,"NaN":1,"N/R":1}


In [29]:
path3="datos/clasificacion_biomedica.xlsx"
df_clasificacion=pd.read_excel(path3)
path4="datos/clasificacion_de_riesgo.xlsx"
df_riesgo=pd.read_excel(path4)
clasificacion_dic = df_clasificacion.set_index('Clasificacion_biomedica').to_dict()["Peso"]
riesgo_dic = df_riesgo.set_index('Clasificacion_de_riesgo').to_dict()["Peso"]
clasificacion_dic["NaN"]=1
riesgo_dic["NaN"]=1



print(clasificacion_dic)
print(riesgo_dic)

{'TRATAMIENTO Y MANTENIMIENTO DE LA VIDA': 3, 'RAHABILITACIÓN': 2, 'DIAGNOSTICO': 2, 'APOYO': 1, 'N/R': 1, 'ANÁLISIS DE LABORATORIO': 1, 'REHABILITACIÓN': 1, 'NaN': 1}
{'I': 1, 'IIA': 2, 'IIB': 3, 'III': 4, 'N/R': 1, 'NaN': 1}


In [30]:
df_salida["Clasificación biomedica"]=df_salida["Clasificación biomedica"].replace(np.nan,"NaN")
df_salida["Clasificación de riesgo"]=df_salida["Clasificación de riesgo"].replace(np.nan,"NaN")

In [31]:
df_salida["Clasificacion ponderada"]=df_salida["Clasificación biomedica"].apply(lambda x: clasificacion_dic[x])
df_salida["Riesgo ponderado"]=df_salida["Clasificación de riesgo"].apply(lambda x: riesgo_dic[x])
df_salida.head()

,Nombre,Descripcion adicional,Marca,Modelo,Estado actual,Servicio,Clasificación biomedica,Clasificación de riesgo,Cantidad de correctivos registrados,Estado ponderado,Servicio ponderado,Clasificacion ponderada,Riesgo ponderado
0,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,Fuera de servicio,QUIROFANO 11,DIAGNOSTICO,IIB,0,1,3.0,2,3
1,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIA,0,1,3.0,2,2
2,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIB,0,1,3.0,2,3
3,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Fuera de servicio,QUIROFANO 1- NEURO,DIAGNOSTICO,IIB,0,1,3.0,2,3
4,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,Fuera de servicio,QUIROFANO 2- NEURO,DIAGNOSTICO,IIB,1,1,3.0,2,3


In [ ]:
def func(x):
    if int(x)<=5:
        return 1
    elif int(x)<=9:
        return 2
    elif int(x)>10:
        return 3
    else:
        return 1

df_salida["Correctivos ponderados"]=df_salida["Cantidad de correctivos registrados"].apply(func)
df_salida.head()


In [ ]:
df_salida["Impacto operacional"]=1
df_salida["Solicitud por el personal"]=1

In [43]:
path5="datos/impacto_operacional.xlsx"
df_impacto=pd.read_excel(path5)
impacto_dic = df_impacto.set_index('Equipo').to_dict()["Peso"]


In [44]:
df_salida["Nombre"]=df_salida["Nombre"].str.upper()
df_salida["Impacto operacional"]=df_salida["Nombre"].apply(lambda x: impacto_dic[x])

In [ ]:
def total(x):
    salida=x[9]*(x[10]*0.211+x[11]*0.033+x[12]*0.128+x[13]*0.06+x[14]*0.208+x[15]*0.359)
    return salida

df_salida["Total"]=df_salida.apply(total,axis=1)
df_salida.head()

In [ ]:
import openpyxl


In [ ]:
wbk=openpyxl.load_workbook("Datos_filtrados.xlsx")
for wks in wbk.worksheets:
    wks.cell(row=1,column=18).value="Total"
wbk.save("Datos_filtrados.xlsx")
wbk.close()

In [ ]:
wbk=openpyxl.load_workbook("Datos_filtrados.xlsx")
wks=wbk.worksheets[0]
maxRow=wks.max_row
for row in range(2,maxRow):
    wks.cell(row=row,column=18).value="=K{}*(L{}*0.211+M{}*0.033+N{}*0.128+O{}*0.06+P{}*0.208+Q{}*0.359)".format(row,row,row,row,row,row,row)
wbk.save("Datos_filtrados.xlsx")
wbk.close()